In [1]:
import pandas as pd
from preprocessing.rating import SecurityRating

In [2]:
bonds = pd.read_csv('../bonds.csv')
bonds_prices = pd.read_csv('../bonds_price_ratings.csv')
rating = SecurityRating(
    bonds=bonds,
)

In [3]:
bonds["rating"] = bonds.apply(rating.calculate_bonds_rating, axis=1)

In [4]:
bonds_with_ratings = bonds.merge(bonds_prices, on="figi", how="inner")

In [5]:
bonds_with_ratings = bonds_with_ratings.rename({"rating_x": "company_rating", "rating_y": "price_rating"}, axis=1)

In [6]:
bonds_with_ratings = bonds_with_ratings[
    ["figi", "sector", "price_rating", "company_rating", "lot", "rub_price"]].dropna()

In [7]:
bonds_with_ratings

,figi,sector,price_rating,company_rating,lot,rub_price
0,TCS00A107D74,industrials,5.398579,6.293961,1,93.86
1,BBG00XH4W3N3,government,3.921603,3.675523,1,56.66
2,BBG00Z8KGGC3,financial,4.403083,5.425456,1,76.96
3,TCS00A102D46,financial,4.281776,5.401428,1,65.40
4,ISSUANCEPRLS,other,6.204494,6.306879,1,95.32
...,...,...,...,...,...,...
1135,BBG012YS2TP2,financial,5.268277,6.204105,1,88.44
1136,TCS00A105HJ9,financial,6.742966,5.442307,1,99.40
1137,BBG00T7HZJG4,materials,0.000000,5.002399,1,0.00
1138,TCS00A104735,real_estate,5.320846,4.801130,1,89.80


In [38]:
bonds_with_ratings["sum_of_ratings"] = bonds_with_ratings["price_rating"] + bonds_with_ratings["company_rating"]

In [40]:
bonds_with_ratings.sort_values(by="sum_of_ratings", ascending=False)

,figi,sector,price_rating,company_rating,lot,rub_price,sum_of_ratings
797,TCS00A106VV3,industrials,5.863216,8.181922,1,99.25,14.045139
721,TCS00A108VW7,materials,5.832779,8.185184,1,97.51,14.017963
739,BBG011DV0155,financial,5.874284,8.136146,1,87.82,14.010430
385,TCS00A107UW1,energy,5.798108,8.190037,1,98.51,13.988145
202,TCS00A1075S4,financial,5.870291,7.933749,1,99.49,13.804040
...,...,...,...,...,...,...,...
323,BBG00NBJSLV4,telecom,0.000000,2.177904,1,0.00,2.177904
356,BBG012G07QT0,financial,0.000000,2.155773,1,0.00,2.155773
101,BBG00GXQKMJ9,government,0.000000,2.153558,1,0.00,2.153558
420,BBG007KHNMS0,financial,0.000000,2.135396,1,0.00,2.135396


In [82]:
bonds_with_ratings.loc[bonds_with_ratings.rub_price <= 100].sort_values(by="sum_of_ratings", ascending=False)

,figi,sector,price_rating,company_rating,lot,rub_price,sum_of_ratings
797,TCS00A106VV3,industrials,5.863216,8.181922,1,99.2500,14.045139
721,TCS00A108VW7,materials,5.832779,8.185184,1,97.5100,14.017963
739,BBG011DV0155,financial,5.874284,8.136146,1,87.8200,14.010430
385,TCS00A107UW1,energy,5.798108,8.190037,1,98.5100,13.988145
202,TCS00A1075S4,financial,5.870291,7.933749,1,99.4900,13.804040
...,...,...,...,...,...,...,...
670,BBG01N8LD6P8,materials,2.288492,4.122222,1,74.9998,6.410714
581,TCS10A109QN4,financial,1.728831,4.588361,1,96.7500,6.317192
368,TCS00A109KX6,financial,1.295787,4.732142,1,92.0000,6.027929
1008,TCS10A108Q29,financial,1.894820,4.091795,1,71.8799,5.986615


In [75]:
import pandas as pd
import math
from ortools.algorithms.python import knapsack_solver
bonds_with_ratings = bonds_with_ratings.loc[bonds_with_ratings.rub_price > 0]

prices = bonds_with_ratings["rub_price"].apply(lambda x: math.ceil(x)).tolist()
ratings = bonds_with_ratings["sum_of_ratings"].apply(lambda x: int(x * 10)).tolist()
capacity = 100

In [76]:
solver = knapsack_solver.KnapsackSolver(
    knapsack_solver.KNAPSACK_DYNAMIC_PROGRAMMING_SOLVER, 'KnapsackBonds'
)

solver.init(ratings, [prices], [capacity])
computed_value = solver.solve()

selected_items = []
for i in range(len(prices)):
    if solver.best_solution_contains(i):
        selected_items.append(bonds_with_ratings.iloc[i])

portfolio_df = pd.DataFrame(selected_items)
print(portfolio_df)

              figi      sector  price_rating  company_rating  lot  rub_price  \
119   TCS70A107SH6    consumer      3.062828        4.512535    1      4.920   
335   TCS10A107U40       other      3.317778        4.512263    1      6.170   
392   TCS10A1050X7    consumer      3.635257        4.571317    1     13.660   
685   TCS70A106U74    consumer      3.391305        4.527559    1      5.700   
775   BBG011MLGP84  government      4.128527        5.972519    1     51.537   
864   TCS20A105AW7       other      3.847246        4.541151    1      6.500   
1083  TCS00A106QS9       other      3.403140        4.528374    1      5.370   

      sum_of_ratings  
119         7.575363  
335         7.830041  
392         8.206574  
685         7.918864  
775        10.101045  
864         8.388396  
1083        7.931514  


In [83]:
computed_value

577

In [77]:
portfolio_df.sort_values(by="sum_of_ratings", ascending=False)

,figi,sector,price_rating,company_rating,lot,rub_price,sum_of_ratings
775,BBG011MLGP84,government,4.128527,5.972519,1,51.537,10.101045
864,TCS20A105AW7,other,3.847246,4.541151,1,6.500,8.388396
392,TCS10A1050X7,consumer,3.635257,4.571317,1,13.660,8.206574
1083,TCS00A106QS9,other,3.403140,4.528374,1,5.370,7.931514
685,TCS70A106U74,consumer,3.391305,4.527559,1,5.700,7.918864
335,TCS10A107U40,other,3.317778,4.512263,1,6.170,7.830041
119,TCS70A107SH6,consumer,3.062828,4.512535,1,4.920,7.575363


In [78]:
import plotly.express as px

In [79]:
px.pie(data_frame=portfolio_df, names='sector')